In [2]:
#Import Pandas for to import data sets from CSV
import pandas as pd

In [40]:
#Importing data sets
enrollments = pd.read_csv('C:/Users/cboat/OneDrive/Documents/Python Scripts/enrollments.csv')
daily_engagement = pd.read_csv('C:/Users/cboat/OneDrive/Documents/Python Scripts/daily_engagement.csv')
daily_engagement_full = pd.read_csv('C:/Users/cboat/OneDrive/Documents/Python Scripts/daily_engagement_full.csv')
project_submissions = pd.read_csv('C:/Users/cboat/OneDrive/Documents/Python Scripts/project_submissions.csv')

In [41]:
# Renaming columns
daily_engagement = daily_engagement.rename(columns = {'acct': 'account_key'}) 

In [42]:
len(enrollments)
print(type(enrollments))
print(enrollments.columns)
print(enrollments.loc[0, 'account_key'])
unique_enrolled_students = set()
for value in enrollments.loc[:, 'account_key']:
    unique_enrolled_students.add(value)
len(unique_enrolled_students)

len(daily_engagement)

unique_engagement_students = set()
for value in daily_engagement.loc[:, 'account_key']:
    unique_engagement_students.add(value)
len(unique_engagement_students)

unique_project_submitters = set()
for value in project_submissions.loc[:, 'account_key']:
    unique_project_submitters.add(value)
len(unique_project_submitters)

<class 'pandas.core.frame.DataFrame'>
Index(['account_key', 'status', 'join_date', 'cancel_date', 'days_to_cancel',
       'is_udacity', 'is_canceled'],
      dtype='object')
448


743

In [43]:
# Check to see Missing Engagement Records

for value in enrollments.loc[:, 'account_key']:
    if value not in unique_engagement_students:
        print(value)
        break

1219


In [58]:
#Checking for more problems in the data. More Students missing?
num_problem_students = 0
print(enrollments.shape[0])
print(enrollments.head(20))

for enrollment in enrollments.loc[:, 'account_key']:
    if (enrollment not in unique_engagement_students and 
            enrollments.loc[enrollment, 'join_date'] != enrollments.loc[enrollment, 'cancel_date']):
        print(enrollment)
        num_problem_students += 1

num_problem_students

1640
    account_key    status   join_date cancel_date  days_to_cancel  is_udacity  \
0           448  canceled  2014-11-10  2015-01-14            65.0        True   
1           448  canceled  2014-11-05  2014-11-10             5.0        True   
2           448  canceled  2015-01-27  2015-01-27             0.0        True   
3           448  canceled  2014-11-10  2014-11-10             0.0        True   
4           448   current  2015-03-10         NaN             NaN        True   
5           448  canceled  2015-01-14  2015-01-27            13.0        True   
6           448  canceled  2015-01-27  2015-03-10            42.0        True   
7           448  canceled  2015-01-27  2015-01-27             0.0        True   
8           448  canceled  2015-01-27  2015-01-27             0.0        True   
9           700  canceled  2014-11-10  2014-11-16             6.0       False   
10          429  canceled  2014-11-10  2015-03-10           120.0       False   
11          429  cancel

63

In [53]:
# Subsetting data

# Create set of items wanting to subset out

udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udactiy_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

# Create function to remove specific data points

def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['accouint_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
return non_udacity_data

non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_engagement_full = remove_udacity_accounts(daily_engagement_full)
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_submission = remove_udacity_accounts(project_submissions)

TypeError: string indices must be integers

In [46]:
#Creating a dictionary os students who didn't cancel or stayed enrolled more than 7 days
#Remember {} creates the dictionary

paid_students = {}

for enrollment in non_udacity_enrollments:
    if (not enrollment['is canceled'] or enrollment['dats_to_cancel'] > 7):
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        if (account_key not in paid_students or
            enrollment_date > paid_student[account_key]):
            paid_student[account_key] = enrollment_date
len(paid_students)

NameError: name 'non_udacity_enrollments' is not defined

In [ ]:
#Finding paid students with the date being within one week of their join date


#Following returns TRUE if the engagemnt date is within one week of the join date for the student
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7 and time_delta.days >= 0

#*******FUNCTION TO SUBSET DATA**********
#Function to find all paid students within the data set
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

#Run the function throughout all of the datasets

paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)
paid_engagement_full = remove_free_trial_cancels(non_udacity_engagement_full)

print len(paid_enrollments)
print len(paid_engagement)
print len(paid_submissions)

In [ ]:
for value in paid_engagement:
    if value['num_courses_visited'] > 0:
        value['has_visited'] = 1
    else:
        value['has_visited'] = 0

In [ ]:
#Creating a new list to subset the students that paid within the first week of engagement
paid_engagement_in_first_week = []
for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']

#If the dates are within one week of each other append them to the empty list created above
    if within_one_week(join_date, engagement_record_date):
         paid_engagement_in_first_week.append(engagement_record)

len(paid_engagement_in_first_week)

In [ ]:
#Creating a list of engagement by accounts 
from collections import defaultdict

engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [49]:
#Creating dictionary to find total minutes spent in the classroom of students.
#This will be used to determine the average amount of minutes spent by students in the classrooms.

total_minutes_by_account = {}

for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record('total_minutes_visited')
    total_minutes_by_account[account_key] = total_minutes

total_minutes = total_minutes_by_account.values()

import numpy as np

np.mean(total_minutes)


NameError: name 'engagement_by_account' is not defined

In [ ]:
student_with_max_minutes = None
max_minutes = 0

for student, total_minutes in total_minutes_by_account.items():
    if total_minutes > max_minutes:
        max_minutes = total_minutes
        student_with_max_minutes = student
max_minutes

In [ ]:
for value in paid_engagement_in_first_week:
    if value['account_key'] == student_with_max_minutes:
        print(value)

In [59]:
from collections import defaultdict

def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        key = data_point[key_name]
        grouped_data[key_name].append(data_point)
    return(grouped_data)

engagement_by_account = grouped_data[paid_engagement_in_first_week, 'account_key']

#******Totaling grouped items. Works like aggreagte in R******
def sum_grouped_items(grouped_data, field_name):
    summed_data = {}

    for key, data_points in grouped_data.items():
        total_classes = 0
        for value in data_points:
            total += value[field_name]
        summed_data[key] = total
    return(summed_data)

total_minutes_by_account = sum_grouped_items(engagement_by_account, 'total_minutes_visited')

import numpy as np

def describe_data(data):
    print(np.mean(data))
    print(np.std(data))
    print(np.min(data))
    print(np.max(data))

total_classes = total_classes_by_account.values()
describe_data(total_minutes)

NameError: name 'engagement_by_account' is not defined

In [ ]:
lessons_completed_by_account = sum_grouped_items(engagement_by_account,
                                                 'lessons_completed')
describe_data(lessons_completed_by_account.values())

In [ ]:
days_visited_by_account = sum_grouped_items(engagement_by_account,
                                            'has_visited')
describe_data(days_visited_by_account.values())

In [ ]:
subway_project_lesson_keys = ['746169184', '3176718735']

pass_subway_project = set()

for value in paid_submissions:
    project = value['lesson_key']
    rating = value['assigned_rating']    

    if ((project in subway_project_lesson_keys) and
            (rating == 'PASSED' or rating == 'DISTINCTION')):
        pass_subway_project.add(value['account_key'])

len(pass_subway_project)

In [ ]:
passing_engagement = []
non_passing_engagement = []

for value in paid_engagement_in_first_week:
    if value['account_key'] in pass_subway_project:
        passing_engagement.append(value)
    else:
        non_passing_engagement.append(value)

print len(passing_engagement)
print len(non_passing_engagement)

In [ ]:
passing_engagement_by_account = group_data(passing_engagement,
                                           'account_key')
non_passing_engagement_by_account = group_data(non_passing_engagement,
                                               'account_key')

print 'non-passing students:'
non_passing_minutes = sum_grouped_items(
    non_passing_engagement_by_account,
    'total_minutes_visited'
)
describe_data(non_passing_minutes.values())
​
print 'passing students:'
passing_minutes = sum_grouped_items(
    passing_engagement_by_account,
    'total_minutes_visited'
)
describe_data(passing_minutes.values())

print 'non-passing students:'
non_passing_lessons = sum_grouped_items(
    non_passing_engagement_by_account,
    'lessons_completed'
)
describe_data(non_passing_lessons.values())

print 'passing students:'
passing_lessons = sum_grouped_items(
    passing_engagement_by_account,
    'lessons_completed'
)
describe_data(passing_lessons.values())

print 'non-passing students:'
non_passing_visits = sum_grouped_items(
    non_passing_engagement_by_account, 
    'has_visited'
)
describe_data(non_passing_visits.values())

print 'passing students:'
passing_visits = sum_grouped_items(
    passing_engagement_by_account,
    'has_visited'
)
describe_data(passing_visits.values())

In [ ]:
import matplotlib as plt
import numpy as np

def describe_data(data):
    print(np.mean(data))
    print(np.std(data))
    print(np.min(data))
    print(np.max(data))
    plt.hist(data)

In [ ]:
#*******ALWYAS USE SEABORN FOR CREATING GRAPHS************

import seaborn as sns

plt.hist(non_passing_visits.values(), bins=5)
plt.xlabel('Number of days')
plt.title('Distribution of classroom visits in the first week for students who do not pass the subway project')

plt.hist(passing_visits.values(), bins=8)
plt.xlabel('Number of days')
plt.title('Distribution of classroom visits in the first week ' + 
          'for students who pass the subway project')